## Step 9: Store the Model

Save the trained model to the Object Storage system configured in your Workbench connection. 

Start by getting the credentials and configuring variables for accessing Object Storage.

In [ ]:
# Imports
import os
from minio import Minio
from minio.error import S3Error

Deserialize from file.

In [ ]:
import torch 

results_train_save_path = "model_train_results.pth"
results_train = torch.load(results_train_save_path)

results_test_save_path = "model_test_results.pth"
results_test = torch.load(results_test_save_path)


In [ ]:
AWS_S3_ENDPOINT_NAME = os.getenv("AWS_S3_ENDPOINT", "").replace('https://', '').replace('http://', '')
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_S3_BUCKET = os.getenv("AWS_S3_BUCKET")

Define the S3 client.

In [ ]:
client = Minio(
    AWS_S3_ENDPOINT_NAME,
    access_key=AWS_ACCESS_KEY_ID,
    secret_key=AWS_SECRET_ACCESS_KEY,
    secure=True
)

Select files to be uploaded (files generated while training and validating the model)

In [ ]:
model_path_train = results_train['save_dir']

model_path_train = results_train['save_dir']
weights_path = os.path.join(model_path_train, "weights")
model_path_test = results_test['save_dir']

files_train = [os.path.join(model_path_train, f) for f in os.listdir(model_path_train) if os.path.isfile(os.path.join(model_path_train, f))]
files_models = [os.path.join(weights_path, f) for f in os.listdir(weights_path) if os.path.isfile(os.path.join(weights_path, f))]
files_test = [os.path.join(model_path_test, f) for f in os.listdir(model_path_test) if os.path.isfile(os.path.join(model_path_test, f))]


Upload the files.

In [ ]:
directory_name= os.path.basename(model_path_train)

for file_path_train in files_train:
    try:
        client.fput_object(AWS_S3_BUCKET, "prototype/pipeline/" + directory_name + "/train-val/" + os.path.basename(file_path_train), file_path_train)
        print(f"'{os.path.basename(file_path_train)}' is successfully uploaded as object to bucket '{AWS_S3_BUCKET}'.")
    except S3Error as e:
        print("Error occurred: ", e)

for file_path_model in files_models:
    try:
        client.fput_object(AWS_S3_BUCKET, "prototype/pipeline/" + directory_name + "/" + os.path.basename(file_path_model), file_path_model)
        print(f"'{os.path.basename(file_path_model)}' is successfully uploaded as object to bucket '{AWS_S3_BUCKET}'.")
    except S3Error as e:
        print("Error occurred: ", e)

for file_path_test in files_test:
    try:
        client.fput_object(AWS_S3_BUCKET, "prototype/pipeline/" + directory_name + "/test/" + os.path.basename(file_path_test), file_path_test)
        print(f"'{os.path.basename(file_path_test)}' is successfully uploaded as object to bucket '{AWS_S3_BUCKET}'.")
    except S3Error as e:
        print("Error occurred: ", e)

## Step 10: Remove local files

Once you uploaded the Model data to the Object Storage, you can remove the local files to save disk space and also the files where we stored the variable values.

In [ ]:
!rm -rf {model_path_train}
!rm -rf {model_path_test}
!rm -rf {results_train_save_path}
!rm -rf {results_test_save_path}